In [12]:
import pandas as pd
import numpy as np
import requests
import bs4
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import re

import warnings
warnings.filterwarnings("ignore")

In [6]:
url_marcas = 'https://xxxxxxxx.com/tabela-fipe-motos'

In [7]:
# Marcas
requisicao_marcas = requests.get(url_marcas)

pagina_marcas = bs4.BeautifulSoup(requisicao_marcas.text, "html.parser")

marcas = pagina_marcas.find_all("a", class_='botao_fake1')


links_marcas = []
#Coletando marcas
for marca in marcas:
    link_brand = str(marca).split('"')[3]
    links_marcas.append(link_brand)
    

In [8]:
##Filtrando marcas

marcas_selecionadas = [r"watts$", r"yamaha$", r"zontes$", r"voltz$", r"triumph$", r"royal-enfield$",
                       r"ktm$", r"kawasaki$", r"kasinski$", r"harley-davidson$", r"honda$", r"husqvarna$",
                       r"ducati$", r"buell$", r"cagiva$", r"dafra$", r"bajaj$", r"bmw$"]

links_marcas = [item for item in links_marcas if any(re.search(regex, item) for regex in marcas_selecionadas)]


In [9]:
#Coletando modelos

df_completo = pd.DataFrame()


for marca in links_marcas:
    requisicao_modelos = requests.get(marca)
    pagina_modelos = bs4.BeautifulSoup(requisicao_modelos.text, "html.parser")
    modelos = pagina_modelos.find_all(class_='modelo_base2')
    marca_nome = marca.split('/')[4]
    
    i=0
    while i < len(modelos):
        modelo = modelos[i]
        modelo = str(modelos[i]).split("\n")[1].strip()
        link_prefix = "https://tabelacarros.com/anos_modelos/motos/"
        link_sufix = f"{marca_nome}/{modelo}"
        link_sufix=link_sufix.replace(" ","-").lower()
        i+=1
        
        df = pd.DataFrame({'Marcas' : marca_nome , 
                           "Modelo":modelo,
                           "link_prefix":link_prefix,
                           "link_sufix":link_sufix,
                                  },index=[i])
        df_completo = pd.concat([df_completo,df])


In [10]:
df_novo = pd.DataFrame()

# Detalhes do modelo através do link inicial
for index, row in df_completo.iterrows():
    link_detalhes = row['link_prefix'] + row['link_sufix']
    requisicao_detalhes = requests.get(link_detalhes)
    pagina_detalhes = bs4.BeautifulSoup(requisicao_detalhes.text, "html.parser")
    modelos_det = pagina_detalhes.find_all("tr", class_='link')
    mes_ref = str(pagina_detalhes.find_all("span", class_="info_legenda")).split("FIPE:")[1].split('</')[0].strip().replace('/','_')
    marca = row['Marcas']
    modelo = row['Modelo'].replace(" ","-")
    imagem = pagina_detalhes.find(class_='box_imagem_carro_200')
    if imagem:
        img_tag = imagem.find('img')
        if img_tag:
            img = img_tag.get('src')

    for linha in modelos_det:
        ano_modelo = linha.find('span', class_='link_ano').text.strip().replace(" ","-")
        link_modelo= f"https://tabelacarros.com/modelo/motos/{marca}/{modelo}/{ano_modelo}"

        # Criar DataFrame com os detalhes
        df = pd.DataFrame({'Marcas': marca,
                           "Modelo": modelo,
                           "link_modelos": link_modelo,
                           "ano/modelo": ano_modelo,
                           "Img": img,
                           "mes_ref": mes_ref}, index=[index])
        df_novo = pd.concat([df_novo, df])

df_novo = df_novo.replace('Zero-Km-a-gasolina', "0 Km")
df_novo= df_novo.drop_duplicates().reset_index()


In [13]:
# Inicializar listas para armazenar os dados
codigos_fipe = []
versoes = []
valores_fipe = []
marcas =[]
modelos= []
ano_modelos = []
imgs = [] 
meses_ref = []
PK =[]

for i in df_novo.index:
    #Trazer Dados
    marca = df_novo.iloc[i]['Marcas']
    modelo = df_novo.iloc[i]['Modelo']
    ano_modelo = df_novo.iloc[i]['ano/modelo']
    img = df_novo.iloc[i]['Img']
    mes_ref = df_novo.iloc[i]['mes_ref']
    
    link = df_novo.iloc[i]['link_modelos']
    requisicao_detalhes = requests.get(link)
    pagina_detalhes = BeautifulSoup(requisicao_detalhes.text, "html.parser")

    # Encontrar a tabela pelo class
    tabela = pagina_detalhes.find("table", class_='tabela1 links')
    if tabela:
#         print("Tabela encontrada")        
        
        # Iterar sobre as linhas do corpo da tabela
        for linha in tabela.find("tbody").find_all("tr"):
            colunas = linha.find_all("td")
            
            # Extrair os dados de cada coluna
            codigo_fipe = colunas[0].find("span", class_="codigo_fipe").text.strip()
            versao = colunas[1].find("span", class_="link_ano").text.strip()
            valor_fipe = colunas[2].find("b").text.strip()
            
            pk = f'{codigo_fipe}_{ano_modelo}_{mes_ref}'
            
            # Adicionar os dados às listas
            PK.append(pk)
            codigos_fipe.append(codigo_fipe)
            versoes.append(versao)
            valores_fipe.append(valor_fipe)
            
            marcas.append(marca)
            modelos.append(modelo)
            ano_modelos.append(ano_modelo)
            imgs.append(img) 
            meses_ref.append(mes_ref)
            
            
    else:
        print("Tabela não encontrada na página:", link)
        
# Criar um DataFrame
dados = {
    "PK": PK,
    "Código FIPE": codigos_fipe,
    "Versão": versoes,
    "Valor Fipe": valores_fipe,
    "Marca" : marcas,
    "Modelos" : modelos,
    "Ano/Modelo" : ano_modelos,
    'url_img' : imgs,
    "MesRef" : meses_ref
}
df = pd.DataFrame(dados)

##################### IMG BIZ #################################
link_biz = "https://tabelacarros.com/imagens/veiculos/motos_anos/tabela_fipe_honda_biz_2019.webp"
df.loc[(df.Marca == "honda") & (df.Modelos == "BIZ"), 'url_img'] = link_biz
##################### IMG BIZ #################################
df
#### Salvar
# nome_arq_dados = f'Moto_Completo_FIPE_{mes_ref}.csv' 
# df.to_csv(nome_arq_dados, index= False)



,PK,Código FIPE,Versão,Valor Fipe,Marca,Modelos,Ano/Modelo,url_img,MesRef
0,837001-0_1997_Junho_2024,837001-0,CHAMPION 45 100cc,R$ 979,bajaj,CHAMPION,1997,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
1,837001-0_1998_Junho_2024,837001-0,CHAMPION 45 100cc,R$ 1.041,bajaj,CHAMPION,1998,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
2,837002-8_1998_Junho_2024,837002-8,CLASSIC 150cc,R$ 1.161,bajaj,CLASSIC,1998,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
3,837004-4_2023_Junho_2024,837004-4,DOMINAR 160,R$ 14.707,bajaj,Dominar-160,2023,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
4,837004-4_2024_Junho_2024,837004-4,DOMINAR 160,R$ 15.583,bajaj,Dominar-160,2024,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
...,...,...,...,...,...,...,...,...,...
4884,895003-2_2024_Junho_2024,895003-2,T 310,R$ 30.359,zontes,T-310,2024,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
4885,895003-2_0 Km_Junho_2024,895003-2,T 310,R$ 32.490,zontes,T-310,0 Km,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
4886,895002-4_2023_Junho_2024,895002-4,V 310,R$ 25.528,zontes,V-310,2023,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024
4887,895002-4_2024_Junho_2024,895002-4,V 310,R$ 27.043,zontes,V-310,2024,https://tabelacarros.com/imagens/veiculos/moto...,Junho_2024


---------------------------------------------------------------------------

---------------------------------------------------------------------------
Appendar à planilha existente Salvar no Google Sheets

In [ ]:
tbl_online = 'Endereço da sua planilha Gsheets'
credjson = 'Credentials.json'
aba = 'nome da aba'

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Caminho para o arquivo JSON de credenciais
credentials_path = credjson

# Defina o escopo das credenciais
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Carregar as credenciais do arquivo JSON
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)

# Autorize o cliente usando as credenciais
client = gspread.authorize(credentials)

# Acesse a planilha
spreadsheet = client.open_by_key(tbl_online)

# Selecione a aba que você deseja atualizar
worksheet = spreadsheet.worksheet(aba)

# Obtenha os valores do DataFrame como uma lista de listas
new_values = df.values.tolist()

# Determine a próxima célula vazia na última linha da planilha
next_row = len(worksheet.get_all_values()) + 1

# Determine a célula de início para adicionar os novos valores
start_cell = "A{}".format(next_row)

# Atualize os valores na planilha a partir da célula de início
worksheet.update(start_cell, new_values)

print("Novos valores mensais adicionados à planilha com sucesso!")


-------------------------------------------------

-------------------------------------------------